## Homework6
Chapter6の講義資料などで扱った購買データを利用して以下の問いに答えて、課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
Chapter6の講義資料の総合問題6-1で求めた最も支持度が高かった商品の2つの組み合わせについて、**リフト値**を求めてください。  
最も支持度が高かった商品の`"StockCode"`を、再度求める必要は無く、コード内で直接記入してください。  
<b>`"StockCode"`のデータは事前に文字列に変換してあります．</b>



提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`をそのまま用いてください。
>- 返り値が数値型になるようにしてください。  

In [1]:
# common
import pandas as pd
import numpy as np

## init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])
online_retail_data_table["StockCode"] = online_retail_data_table["StockCode"].astype(str) 

# 数字があるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [12]:
target_online_retail_data_tb["StockCode"].value_counts().head(2)

85123A    2035
22423     1724
Name: StockCode, dtype: int64

In [15]:
stockcode_counts = target_online_retail_data_tb['StockCode'].value_counts()
stockcode_counts

85123A    2035
22423     1724
85099B    1618
84879     1408
47566     1397
          ... 
90168        1
90169        1
90202A       1
90118        1
23843        1
Name: StockCode, Length: 3665, dtype: int64

In [17]:
stockcodes_over_1000 = stockcode_counts[stockcode_counts > 1000].index
stockcodes_over_1000.tolist()

['85123A',
 '22423',
 '85099B',
 '84879',
 '47566',
 '20725',
 '22720',
 '20727',
 'POST',
 '23203',
 '22383',
 '21212',
 '22197',
 '23209',
 '23298',
 '22086',
 '22382']

In [18]:
from itertools import combinations
def homework(target_online_retail_data_tb):
    # レコード数が1000より多い商品のリストを作成
    popular_items = target_online_retail_data_tb['StockCode'].value_counts().loc[lambda x: x > 1000].index.tolist()
    # ペア（キー）に対する支持率
    support_dict = {}
    # 個々に対する支持率
    support_items = {}
    # 全てのトランザクション数
    trans_all = len(set(target_online_retail_data_tb.InvoiceNo))
    for item in popular_items:
        trans_i = len(set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode'] == item].InvoiceNo))
        # itemをキーとして支持率を格納
        support_items[item] = trans_i / trans_all

    for pair in combinations(popular_items, 2):
        item_a, item_b = pair
        trans_a = set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode'] == item_a].InvoiceNo)
        trans_b = set(target_online_retail_data_tb[target_online_retail_data_tb['StockCode'] == item_b].InvoiceNo)
        trans_ab = trans_a & trans_b
        support_ab = len(trans_ab) / trans_all
        support_dict[pair] = support_ab

    # 支持度が最大となる組み合わせを求める
    max_support_pair = max(support_dict, key=support_dict.get)
    max_support = support_dict[max_support_pair]

    # リフト値を計算
    # support_itemsにはそれぞれの商品の支持率が格納されているので、それらを取り出して、リフト値を計算
    my_result = max_support / (support_items[max_support_pair[0]] * support_items[max_support_pair[1]])

    return my_result


**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).